In [2]:
import pandas as pd
import statsmodels.api as sm

# Завантажуємо вбудований макроекономічний датасет
df = sm.datasets.macrodata.load_pandas().data
#
# Функція для створення дати з року та кварталу
def create_date(row):
    return pd.Timestamp(year=int(row['year']), month=int(row['quarter'] * 3 - 2), day=1)

# Застосовуємо функцію для створення нового стовпця з датою
df['date'] = df.apply(create_date, axis=1)


import numpy as np
import matplotlib.pyplot as plt
from optimizers import GDOptimizer
from model import NN
from layers import DenseLayer
from activations import ReLU
from losses import MSE
from sklearn.preprocessing import StandardScaler

np.random.seed(42) 

# Перетворюємо дані в масиви
# X = df_selected[['m1', 'realinv', 'unemp']].values  # Вхідні змінні
# y = df_selected['cpi'].values  # Цільова змінна
year = df.drop_duplicates("year")
# year = year.tail(80)[::-1]

X = year[['realgdp', 'realcons', 'realinv', 'realgovt',
        'realdpi', 'cpi', 'm1', 'tbilrate', 'unemp', 'pop', 'infl', 'realint']].values  # Вхідні змінні
y = year['cpi'].values  # Цільова змінна

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [7]:
import numpy as np

import numpy as np
import typing as tp
from datetime import datetime

class BaseOptimizer:
    def apply(self, *args, **kwargs):
        raise NotImplementedError("This method should be overridden by subclasses")

class GeneticAlgorithmOptimizer(BaseOptimizer):
    """
    `Vanilla` Genetic Algorithm.
    """

    def __init__(self, num_population: int, k: int = 5, **kwargs):
        self._num_population = num_population
        self._k = k
        self._population = None
        self._best_iter = None
        self._last_score = np.inf
        self._iter = 0
        self._tol = kwargs.pop("tol", 1e-2)
        super().__init__(**kwargs)

    @staticmethod
    def construct_genome(W: np.ndarray, weight_init: tp.Callable[..., np.ndarray]):
        return 0.1 * weight_init(0, 1, size=W.shape)

    @staticmethod
    def crossover(ind_1: np.ndarray, ind_2: np.ndarray) -> tp.Tuple[np.ndarray, np.ndarray]:
        assert len(ind_1) == len(ind_2), "individuals must have same length"
        index = np.random.default_rng().integers(len(ind_1))
        ind_12 = np.concatenate((ind_1[:index], ind_2[index:]), axis=None)
        ind_21 = np.concatenate((ind_2[:index], ind_1[index:]), axis=None)
        return ind_12, ind_21

    @staticmethod
    def mutate(ind: np.ndarray, mu: float = 0.1, sigma: float = 1.0, factor: float = 0.01) -> np.ndarray:
        seed = int(datetime.utcnow().timestamp() * 1e5)
        ind += factor * np.random.default_rng(seed).normal(loc=mu, scale=sigma, size=len(ind))
        return ind

    def apply(
        self,
        loss: tp.Callable[[np.ndarray, np.ndarray], float],
        input_tensor: np.ndarray,
        output_tensor: np.ndarray,
        W: np.ndarray,
        **kwargs,
    ) -> tp.Tuple[bool, np.ndarray, float]:
        verbose = kwargs.pop("verbose", False)
        seed = int(datetime.utcnow().timestamp() * 1e5)
        to_stop = False
        
        if self._population is None:
            self._population = [
                self.construct_genome(W, np.random.default_rng(seed + 42 * i).normal)
                for i in range(self._num_population)
            ]
        else:
            population = self._population[:]
        
        scores = [loss(g, input_tensor, output_tensor) for g in population]
        scores, scores_idx = np.sort(scores), np.argsort(scores)
        
        if verbose:
            print(f"Best individual - {scores[0]}")

        if scores[0] < self._tol:
            to_stop = True
        
        self._population = np.array(population)[scores_idx][:self._num_population - self._k * 3].tolist()
        probas = 1.0 - (scores - np.min(scores)) / np.ptp(scores)
        probas /= sum(probas)

        for _ in range(self._k):
            indices = np.random.default_rng(seed).choice(scores_idx, 2, p=probas)
            ind_1, ind_2 = self.crossover(population[indices[0]], population[indices[1]])
            self._population.append(self.mutate(ind_1, factor=np.random.default_rng(seed).normal(0.01, 0.1)))
            self._population.append(self.mutate(ind_2, factor=np.random.default_rng(seed).normal(0.01, 0.1)))

        idx_survived = np.random.default_rng().choice(scores_idx[:len(population)], self._k)
        for idx in idx_survived:
            self._population.append(self.mutate(population[idx], factor=np.random.default_rng(seed).normal(0.1, 0.1)))

        self._iter += 1
        return to_stop, np.array(population[scores_idx[0]]), scores[0]

# Example of how to use it
X_scaled = scaler.fit_transform(X)
y = y.reshape(-1, 1)

model = NN(X_scaled.shape[1])
model.add_layer(DenseLayer(8, ReLU()))
model.add_layer(DenseLayer(1, ReLU()))

ga_optimizer = GeneticOptimizer(generations=50)
ga_optimizer.evolve(model, X_scaled, y)

# Make predictions after GA optimization
predictions = model.forward(X_scaled)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()